In [36]:
from docxtpl import * 
from typing import Dict
import sys
sys.path.insert(0, '/models')
import models

# Site references

In [37]:
#cspell:disable


#init proyect
from models.components import Tech
from models.geometry import GeoPosition
from models.inventory import Building, Project

project:Project = Project(
    building = Building(
        geolocation=GeoPosition(latitude=-33,longitude=-70),
        name='Cesfam Pimpim',
        address='calle falsa 123',
        city='Valparaíso'),
    technology= [Tech.PHOTOVOLTAIC]
    ) 

ModuleNotFoundError: No module named 'consumption'

In [3]:
project.weather.get_data().sample(5)

,date,T2M,ALLSKY_SFC_SW_DNI,ALLSKY_SFC_SW_DIFF,ALLSKY_SRF_ALB,SZA,WS10M,year,month,day,hour
5,2023-01-01 00:00:00,-5.18,0.00,0.00,None,None,0.79,2023,01,01,00
26,2023-01-01 21:00:00,-3.13,0.00,0.00,None,None,1.45,2023,01,01,21
44,2023-01-02 15:00:00,7.01,611.77,212.16,0.13,50.87,6.58,2023,01,02,15
12,2023-01-01 07:00:00,3.77,754.67,150.46,0.12,55.73,4.86,2023,01,01,07
35,2023-01-02 06:00:00,2.95,777.72,99.09,0.13,68.25,2.69,2023,01,02,06


In [2]:
from models.photovoltaic import Photovoltaic


panel_module = Photovoltaic(weather=project.weather,power=250,quantity=1)
energy_gen = panel_module.get_energy()
energy_gen.sample(5)

ModuleNotFoundError: No module named 'geometry'